In [1]:
from zeno import zeno
import pandas as pd

df = pd.read_csv("metadata.csv")

zeno({
    "metadata": df,
    "view": "audio-transcription",
    "data_path": "./audio-recordings/",
    "data_column": "id"
})

RuntimeError: Directory '/Users/acabrera/dev-research/22-zeno/zeno/examples/speech-accent/audio-recordings' does not exist

In [ ]:
from zeno import zeno
import math
import os
import pandas as pd

import torch
import whisper
from jiwer import wer
from zeno import ZenoOptions, distill, metric, model
import librosa
import numpy as np
from zeno import ZenoOptions, distill

In [ ]:
df = pd.read_csv("metadata.csv")

In [ ]:
zeno({
    "metadata": df,
    "view": "audio-transcription",
    "data_path": "/Users/acabrera/dev/data/speech-accent-archive/recordings/recordings/",
    "data_column": "id"
})

In [ ]:
@model
def load_model(model_path):
    if "sst" in model_path:
        device = torch.device("cpu")
        model, decoder, utils = torch.hub.load(
            repo_or_dir="snakers4/silero-models",
            model="silero_stt",
            language="en",
            device=device,
        )
        (read_batch, _, _, prepare_model_input) = utils

        def pred(df, ops: ZenoOptions):
            files = [os.path.join(ops.data_path, f) for f in df[ops.data_column]]
            input = prepare_model_input(read_batch(files), device=device)
            return [decoder(x.cpu()) for x in model(input)]

        return pred

    elif "whisper" in model_path:
        model = whisper.load_model("tiny")

        def pred(df, ops: ZenoOptions):
            files = [os.path.join(ops.data_path, f) for f in df[ops.data_column]]
            outs = []
            for f in files:
                outs.append(model.transcribe(f)["text"])
            return outs

        return pred


@distill
def country(df, ops: ZenoOptions):
    if df["0birthplace"][0] == df["0birthplace"][0]:
        return df["0birthplace"].str.split(", ")[-1][-1]
    return ""


@distill
def wer_m(df, ops: ZenoOptions):
    return df.apply(lambda x: wer(x[ops.label_column], x[ops.output_column]), axis=1)


@metric
def avg_wer(df, ops: ZenoOptions):
    avg = df[ops.distill_columns["wer_m"]].mean()
    if math.isnan(avg):
        return 0
    return avg

@distill
def amplitude(df, ops: ZenoOptions):
    files = [os.path.join(ops.data_path, f) for f in df[ops.data_column]]
    amps = []
    for audio in files:
        y, _ = librosa.load(audio)
        amps.append(float(np.abs(y).mean()))
    return amps


@distill
def length(df, ops: ZenoOptions):
    files = [os.path.join(ops.data_path, f) for f in df[ops.data_column]]
    amps = []
    for audio in files:
        y, _ = librosa.load(audio)
        amps.append(len(y))
    return amps



In [ ]:
zeno({
    "metadata": df,
    "functions": [load_model, country, wer_m, avg_wer, amplitude, length],
    "view": "audio-transcription",
    "models": ["silero_sst", "whisper"],
    "data_path": "/Users/acabrera/dev/data/speech-accent-archive/recordings/recordings/",
    "data_column": "id",
    "label_column": "label",
    "samples": 10,
    "port": 8008,
})
# metadata = "metadata.csv"
# # data_path = "https://zenoml.s3.amazonaws.com/accents/"